In [105]:
import requests
import pandas as pd
from IPython.display import display

In [106]:
API_TOKEN = "SG_APIM_FNS4ADJYW4ZJSET806PQT78DT8DDGAHSP2B2TKVF0K3V2ZG6VYAG"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

In [107]:
zip_code = "2500"
stores_url = f"https://api.sallinggroup.com/v1/food-waste/?zip={zip_code}"
store_response = requests.get(stores_url, headers=headers)

all_rows = []

if store_response.status_code == 200:
    stores = store_response.json()

    for store_entry in stores:
        store_info = store_entry.get("store", {})
        store_id = store_info.get("id")
        store_name = store_info.get("name")

        if not store_id:
            continue

        clearance_url = f"https://api.sallinggroup.com/v1/food-waste/{store_id}"
        clearance_response = requests.get(clearance_url, headers=headers)

        if clearance_response.status_code == 200:
            data = clearance_response.json()
            clearances = data.get("clearances", [])

            for item in clearances:
                product = item.get("product", {})
                offer = item.get("offer", {})

                all_rows.append({
                    "Store": store_name,
                    "Product": product.get("description"),
                    "Category (da)": product.get("categories", {}).get("da", ""),
                    "Price (DKK)": offer.get("newPrice"),
                    "Original Price": offer.get("originalPrice"),
                    "Discount": offer.get("discount"),
                    "Stock": offer.get("stock"),
                })

In [108]:
df = pd.DataFrame(all_rows)

In [109]:
df.to_csv("discounted_products_full.csv", index=False)
print("✅ Table exported to 'discounted_products_full.csv'")

✅ Table exported to 'discounted_products_full.csv'


In [110]:
from deep_translator import GoogleTranslator
import pandas as pd

In [111]:
df = pd.read_csv("discounted_products_full.csv") # load discounted_products_full 

In [112]:
translator = GoogleTranslator(source='auto', target='en') # Translate 

df['Product_EN'] = df['Product'].apply(lambda x: translator.translate(x) if pd.notna(x) else "")
df['Category_EN'] = df['Category (da)'].apply(lambda x: translator.translate(x) if pd.notna(x) else "")

In [113]:
df.to_csv("discounted_products_translated.csv", index=False)
print("✅ Translation complete. Saved to 'discounted_products_translated.csv'") 

✅ Translation complete. Saved to 'discounted_products_translated.csv'


In [128]:
import pandas as pd
import re
from gensim.models import Word2Vec

In [130]:
xfile_path = "discounted_products_translated.csv"  
df = pd.read_csv(file_path)

In [131]:
# clean the data
category_col = "Category_EN" if "Category_EN" in df.columns else "Category (da)"
df[category_col] = df[category_col].fillna("").str.lower()
df["Product_EN"] = df["Product_EN"].fillna("").str.lower()
df["Product"] = df["Product"].fillna("").str.lower()

In [132]:
# nested category values 
def map_category(val):
    if pd.isna(val) or val.strip() == "":
        return "others"
    return val.split(">")[0].strip() if ">" in val else val.strip()

df["Mapped_Category"] = df[category_col].apply(map_category)

In [133]:
# category-to-product mapping
unique_categories = df["Mapped_Category"].unique()
category_mapping = {cat: df[df["Mapped_Category"] == cat] for cat in unique_categories if cat.strip() != ""}

In [134]:
# product-to-category mapping
product_to_category = dict(zip(df["Product_EN"], df["Mapped_Category"]))
product_to_category.update(dict(zip(df["Product"], df["Mapped_Category"])))

In [135]:
output_columns = [col for col in ["Product_EN", "Original Price", "Discounted Price (DKK)", "Store"] if col in df.columns]

In [141]:
while True:
    user_input = input("\nType a product name to find similar discounted products (or 'exit' to stop): ").lower().strip()

    if user_input == "exit":
        break
    
    # Partial match in product name
    keyword_df = df[df["Product_EN"].str.contains(user_input, case=False, na=False)]
    if not keyword_df.empty:
        print(f"\nProducts matching '{user_input}':\n")
        print(keyword_df[output_columns].to_string(index=False))
        continue

    # Partial match in category
    partial_categories = [cat for cat in category_mapping if user_input in cat]
    if partial_categories:
        for cat in partial_categories:
            result_df = category_mapping[cat][output_columns].copy()
            if not result_df.empty:
                print(f"\nDiscounted products under category '{cat}':\n")
                print(result_df.to_string(index=False))
        continue

    # Exact product-to-category lookup
    if user_input in product_to_category:
        product_category = product_to_category[user_input]
        category_df = category_mapping.get(product_category, pd.DataFrame())
        if not category_df.empty:
            result_df = category_df[output_columns].copy()
            print(f"\n'{user_input}' belongs to category '{product_category}'.\nHere are all discounted items in that category:\n")
            print(result_df.to_string(index=False))
        else:
            print(f"No other discounted products found in '{user_input}'.")
    else:
        print(f"'{user_input}' not recognized. Try another.")



Products matching 'chicken':

                  Product_EN  Original Price  Price (DKK)                    Store
  wrap w/chicken/curry named           22.95        18.93 Netto Valby, Roskildevej
       chicken burrito named           30.00        23.00 Netto Valby, Roskildevej
chicken breast oak happiness           13.75        10.00    Netto Toftegårds Allé
  wrap w/chicken/curry named           22.95        16.00 Netto Valby Maskinfabrik

Products matching 'ost':

                  Product_EN  Original Price  Price (DKK)                    Store
oliven/salatost the campaign           19.95         11.0 Netto Valby, Roskildevej
 grovhak. leverposty stryhns           29.95         15.0    Netto Toftegårds Allé
         salatost block puck           25.00         19.0 Netto Valby Maskinfabrik
         skiveost grønnelund           45.00         25.0 Netto Valby Maskinfabrik
    frankf. ost c. vollstedt           25.00         19.0 Netto Valby Maskinfabrik
 skiveost l 45+ h.c. anderse 